In [1]:
def calcLoss(original_data, processed_data):
    loss = zip(original_data, processed_data)
    loss = list(map(lambda x: x[0] - x[1], loss))
    loss = list(map(lambda x: reduce(lambda a, b: a + b, x), loss))
    loss = reduce(lambda a, b: a + b, loss)
    return abs(loss)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import decomposition
import librosa
from os import listdir
from os.path import isfile, join
from sklearn import datasets
import soundfile as sf
from functools import reduce

In [3]:
in_filepath = "data1/"
out_filepath = "out/"

filenames = [f for f in listdir(in_filepath) if isfile(join(in_filepath, f))]
data = list(map(lambda filename: in_filepath + filename, filenames))
data = list(map(lambda filename: librosa.load(filename)[0], data))
sr = librosa.load(in_filepath + filenames[0])[1]
data = zip(data, filenames)
data = list(filter(lambda x: len(x[0]) > 110000, data))
filenames = data
data = list(map(lambda x: x[0], data))
filenames = list(map(lambda x: x[1], filenames))
data = list(map(lambda x: x[:110000], data))
data = np.array(list(map(lambda x: x.tolist(), data)))

/home/willy/.local/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/willy/.local/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/willy/.local/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/willy/.local/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/willy/.local/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/will

/home/willy/.local/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/willy/.local/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(110000, 1000)
        self.fc2 = nn.Linear(1000, 200)
        self.fc3 = nn.Linear(200, 43)
        self.fc4 = nn.Linear(43, 200)
        self.fc5 = nn.Linear(200, 1000)
        self.fc6 = nn.Linear(1000, 110000)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x

net = Net().cuda()
print(net)

Net(
  (fc1): Linear(in_features=110000, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=43, bias=True)
  (fc4): Linear(in_features=43, out_features=200, bias=True)
  (fc5): Linear(in_features=200, out_features=1000, bias=True)
  (fc6): Linear(in_features=1000, out_features=110000, bias=True)
)


In [5]:
data = torch.from_numpy(data).cuda()

In [6]:
input = data
output = net(input.float())
target = input

In [7]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [9]:
num_epochs = 100000
epoch = 0
while epoch < num_epochs:
    optimizer.zero_grad()
    output = net(input.float())
    loss = criterion(output, target.float())
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(str(epoch) + " | " + str(calcLoss(input.data.cpu().numpy(), output.data.cpu().numpy())))
    epoch += 1

0 | 117.51436190089568
100 | 117.2412428317382
200 | 116.9685867253154
300 | 116.6969193295106
400 | 116.42593412888688
500 | 116.15448417314089
600 | 115.88330186310647
700 | 115.61378495983242
800 | 115.34489035301318
900 | 115.07654581944259
1000 | 114.80908416805848
1100 | 114.54132224853198
1200 | 114.2742472133266
1300 | 114.00751373480468
1400 | 113.74210871611082
1500 | 113.4775412054512
1600 | 113.21328921820147
1700 | 112.95044171220776
1800 | 112.68741592461608
1900 | 112.42541751650948
2000 | 112.16459827207088
2100 | 111.9040939164695
2200 | 111.64297509835866
2300 | 111.38297480735834
2400 | 111.1238436751171
2500 | 110.86483738034418
2600 | 110.60623482502321
2700 | 110.3479592325
2800 | 110.09012011161009
2900 | 109.83290976282457
3000 | 109.57678388690489
3100 | 109.32210253383325
3200 | 109.06877094333846
3300 | 108.81610093862197
3400 | 108.56514128875803
3500 | 108.31512144880413
3600 | 108.06589282336951
3700 | 107.8173297579249
3800 | 107.56882444840245
3900 | 107

31900 | 57.300071839335715
32000 | 57.16724055220925
32100 | 57.03471647871788
32200 | 56.90235992214161
32300 | 56.77030741280555
32400 | 56.638808535180715
32500 | 56.5077414389541
32600 | 56.37711999869293
32700 | 56.24693486936826
32800 | 56.11686550776517
32900 | 55.98696381908462
33000 | 55.85741009708158
33100 | 55.72803855005054
33200 | 55.598730227405895
33300 | 55.469701237863546
33400 | 55.34091366800544
33500 | 55.212407738546744
33600 | 55.08397596139707
33700 | 54.95611042347605
33800 | 54.828421066760484
33900 | 54.70104261792471
34000 | 54.57353869133908
34100 | 54.44603531095595
34200 | 54.31863858976135
34300 | 54.191597000922
34400 | 54.06512023357459
34500 | 53.93944666314451
34600 | 53.81402638164286
34700 | 53.688800301952824
34800 | 53.56393390771768
34900 | 53.439169281394015
35000 | 53.31453081011364
35100 | 53.190144717509384
35200 | 53.066027470403924
35300 | 52.94208776107348
35400 | 52.8184368262298
35500 | 52.69481975105714
35600 | 52.571458607084196
35700

63000 | 26.86955602824742
63100 | 26.79929699378477
63200 | 26.729109099949486
63300 | 26.659179541795087
63400 | 26.58931755603385
63500 | 26.51957489187021
63600 | 26.44993337316578
63700 | 26.380327415278547
63800 | 26.310990401539815
63900 | 26.241683407691884
64000 | 26.172572032997554
64100 | 26.103824899108762
64200 | 26.03520133022062
64300 | 25.966722015976078
64400 | 25.89846423779341
64500 | 25.830266284253188
64600 | 25.76227574603013
64700 | 25.694207814714677
64800 | 25.626074652689343
64900 | 25.558009744822588
65000 | 25.490127327349686
65100 | 25.42254300754553
65200 | 25.355260201120704
65300 | 25.288143991210067
65400 | 25.22115883351785
65500 | 25.154028961558883
65600 | 25.086757145421647
65700 | 25.019542209790192
65800 | 24.95247288653241
65900 | 24.88554586740179
66000 | 24.818590977872876
66100 | 24.751851825581756
66200 | 24.68524403406605
66300 | 24.618673475723202
66400 | 24.551984001881696
66500 | 24.485490904468183
66600 | 24.419172840199057
66700 | 24.353

93800 | 9.844675750351374
93900 | 9.801869945000139
94000 | 9.758981397229721
94100 | 9.716023248325431
94200 | 9.673226524999112
94300 | 9.63044755198985
94400 | 9.587944946637212
94500 | 9.545470703216555
94600 | 9.503138062830434
94700 | 9.460877866122022
94800 | 9.418741417413688
94900 | 9.376670179414536
95000 | 9.33463684206868
95100 | 9.292548903705896
95200 | 9.250511258647524
95300 | 9.20855574120083
95400 | 9.166703066812037
95500 | 9.124879046340315
95600 | 9.083035740680092
95700 | 9.041559651140009
95800 | 9.000160148303397
95900 | 8.958799847741606
96000 | 8.917436776335084
96100 | 8.876280128926513
96200 | 8.835017095716243
96300 | 8.79371945813978
96400 | 8.752618479158404
96500 | 8.711589959324332
96600 | 8.67052469026908
96700 | 8.629318647668473
96800 | 8.588100801814793
96900 | 8.546763686542139
97000 | 8.505589565988661
97100 | 8.464559299543684
97200 | 8.423620718989472
97300 | 8.382679248155686
97400 | 8.341710916287187
97500 | 8.300847664606184
97600 | 8.2601238

In [10]:
num_epochs = 1000000
epoch = 0
while epoch < num_epochs:
    optimizer.zero_grad()
    output = net(input.float())
    loss = criterion(output, target.float())
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(str(epoch) + " | " + str(calcLoss(input.data.cpu().numpy(), output.data.cpu().numpy())))
    epoch += 1

0 | 7.303307047508272
100 | 7.264276387568091
200 | 7.225339034299171
300 | 7.186423669926401
400 | 7.147652608017554
500 | 7.108908088165438
600 | 7.070254591723599
700 | 7.031592228416915
800 | 6.99291420092118
900 | 6.954346928215958
1000 | 6.9158318510487815
1100 | 6.877479202427597
1200 | 6.83934617991213
1300 | 6.801308119134549
1400 | 6.763342201740445
1500 | 6.725477433932386
1600 | 6.687732590203953
1700 | 6.650135416229889
1800 | 6.61260171630943
1900 | 6.575087610528274
2000 | 6.53762466354033
2100 | 6.50012126291563
2200 | 6.462754007754259
2300 | 6.4254998791636835
2400 | 6.388299255313316
2500 | 6.3513451796950475
2600 | 6.314283708839642
2700 | 6.2772494997532275
2800 | 6.240352766917343
2900 | 6.2034066711801215
3000 | 6.166544310862949
3100 | 6.129744338884623
3200 | 6.092978238366189
3300 | 6.056122762829321
3400 | 6.019349994490542
3500 | 5.982646969174087
3600 | 5.946012071902945
3700 | 5.909509861657855
3800 | 5.872868600088398
3900 | 5.836241529554857
4000 | 5.799

31300 | 2.2275751341496735
31400 | 2.249104528520774
31500 | 2.2705432453806216
31600 | 2.2919516072436172
31700 | 2.3133231581798546
31800 | 2.3347407381900203
31900 | 2.35615960736527
32000 | 2.3775221490016056
32100 | 2.398888547570322
32200 | 2.4202957510535716
32300 | 2.4415914900566795
32400 | 2.4628282520718576
32500 | 2.4839733196024545
32600 | 2.505163854817737
32700 | 2.526315962295711
32800 | 2.547467622760677
32900 | 2.5686200008115057
33000 | 2.589602053758653
33100 | 2.61056439772642
33200 | 2.631450715311302
33300 | 2.6520488908429334
33400 | 2.672579113112305
33500 | 2.6929653313557074
33600 | 2.713373342170825
33700 | 2.7337212471368266
33800 | 2.7540582321091724
33900 | 2.7741261115436364
34000 | 2.7940627943910377
34100 | 2.8138011420398916
34200 | 2.8335963122765397
34300 | 2.8533801931573626
34400 | 2.8733377022774667
34500 | 2.893246736644186
34600 | 2.91285386916229
34700 | 2.9322006365903226
34800 | 2.9514413035869884
34900 | 2.9706221419367003
35000 | 2.9898277

62700 | 7.062855908089658
62800 | 7.074497013215703
62900 | 7.085974462440161
63000 | 7.097400612819682
63100 | 7.108784059479374
63200 | 7.120135685356443
63300 | 7.131463292349734
63400 | 7.142665203643672
63500 | 7.15382571723171
63600 | 7.164953615406601
63700 | 7.176156861804655
63800 | 7.18717764409689
63900 | 7.198138745137383
64000 | 7.209108979161327
64100 | 7.22010787634599
64200 | 7.231137034293395
64300 | 7.242195738295093
64400 | 7.253271178668227
64500 | 7.264368109845462
64600 | 7.275375955591015
64700 | 7.28644029969132
64800 | 7.297375026488451
64900 | 7.308374034282904
65000 | 7.319435623307298
65100 | 7.330387560127063
65200 | 7.341218530543527
65300 | 7.352050872380685
65400 | 7.362767450305226
65500 | 7.373425863960845
65600 | 7.384133395966499
65700 | 7.394885538333481
65800 | 7.405543469247503
65900 | 7.416101801377179
66000 | 7.426674763901472
66100 | 7.437171966403865
66200 | 7.447573370759981
66300 | 7.4578954916380535
66400 | 7.467936204913938
66500 | 7.47784

94400 | 9.773281576544147
94500 | 9.781303857999998
94600 | 9.789250764115646
94700 | 9.797072432656247
94800 | 9.80486000397627
94900 | 9.812726664516727
95000 | 9.820543115405368
95100 | 9.828428220098655
95200 | 9.836307475996133
95300 | 9.84418049213468
95400 | 9.852078514906449
95500 | 9.859903772881577
95600 | 9.86773858513811
95700 | 9.875669412161411
95800 | 9.883636047295337
95900 | 9.891615563817364
96000 | 9.899485659261924
96100 | 9.907301424842668
96200 | 9.915217987311049
96300 | 9.923287896579719
96400 | 9.931630565186648
96500 | 9.939989597787868
96600 | 9.948313979921519
96700 | 9.95651292076877
96800 | 9.964706406729643
96900 | 9.972909008517714
97000 | 9.981156189188592
97100 | 9.989404376540957
97200 | 9.997707648765068
97300 | 10.005972262308429
97400 | 10.01419729629674
97500 | 10.022438370498318
97600 | 10.030660401272367
97700 | 10.038857195503848
97800 | 10.047148881513973
97900 | 10.055402600137537
98000 | 10.063554581234936
98100 | 10.071505000014664
98200 | 

124200 | 12.044276201839917
124300 | 12.052694293136252
124400 | 12.060958496391091
124500 | 12.069235859258466
124600 | 12.07755733398039
124700 | 12.085951986967071
124800 | 12.094418457777234
124900 | 12.102886900935797
125000 | 12.111340425690056
125100 | 12.119712201521676
125200 | 12.127964227820254
125300 | 12.136162882654158
125400 | 12.144364339395676
125500 | 12.15256974019007
125600 | 12.16081207154252
125700 | 12.16900423842321
125800 | 12.177184958127551
125900 | 12.185228649487495
126000 | 12.193345563147883
126100 | 12.201487765018097
126200 | 12.209609627622964
126300 | 12.217680667249393
126400 | 12.22581402662058
126500 | 12.23393286535656
126600 | 12.242095625580983
126700 | 12.250176456167988
126800 | 12.258195178779939
126900 | 12.266124389447437
127000 | 12.274041289247858
127100 | 12.281995383549425
127200 | 12.289897611171224
127300 | 12.29776051250178
127400 | 12.305730229614936
127500 | 12.313727929660898
127600 | 12.321658404807065
127700 | 12.329530874023702

...

867000 | 19.489886303786196
867100 | 19.489530823554677
867200 | 19.48916502007933
867300 | 19.488812776503327
867400 | 19.488469613067313
867500 | 19.48811672519196
867600 | 19.48776121398942
867700 | 19.487408837708248
867800 | 19.487039805204635
867900 | 19.486702668888714
868000 | 19.48635267359737
868100 | 19.48602281454328
868200 | 19.48569192863147
868300 | 19.485345561856096
868400 | 19.485017411976745
868500 | 19.484686770697184
868600 | 19.484354839067468
868700 | 19.484012477790888
868800 | 19.483670645400938
868900 | 19.483327730671366
869000 | 19.48299823371713
869100 | 19.482668290191445
869200 | 19.48233516214555
869300 | 19.481988893366864
869400 | 19.48165547293434
869500 | 19.481319833608044
869600 | 19.480949374781407
869700 | 19.48061920812302
869800 | 19.480293231673162
869900 | 19.47996149654114
870000 | 19.479612463904463
870100 | 19.47927169527998
870200 | 19.47894392682179
870300 | 19.478598126688443
870400 | 19.47826154762129
870500 | 19.477913545147434
870600

896800 | 19.37588211003754
896900 | 19.37540207285861
897000 | 19.374936641210073
897100 | 19.374473807877852
897200 | 19.374008084726714
897300 | 19.37354079599641
897400 | 19.373068818256574
897500 | 19.37259342242601
897600 | 19.3721083146902
897700 | 19.371622184016818
897800 | 19.371107979183083
897900 | 19.370615788345507
898000 | 19.370100911750118
898100 | 19.369591721233295
898200 | 19.3690932462157
898300 | 19.36860744973615
898400 | 19.36809897682429
898500 | 19.367589173075206
898600 | 19.367082213600273
898700 | 19.36659224018445
898800 | 19.3660826145308
898900 | 19.365544284485853
899000 | 19.365039502406265
899100 | 19.364535610278157
899200 | 19.36404029721712
899300 | 19.363529333302044
899400 | 19.363033939476058
899500 | 19.36255400473988
899600 | 19.362059161647498
899700 | 19.361574636419622
899800 | 19.361075401485554
899900 | 19.360555412162515
900000 | 19.360038970439977
900100 | 19.359514055597405
900200 | 19.358989780814458
900300 | 19.358458665529955
900400 

926500 | 19.22355641773652
926600 | 19.223164650027005
926700 | 19.222812711998134
926800 | 19.222435696294955
926900 | 19.22207333906295
927000 | 19.221690013105448
927100 | 19.2213131387764
927200 | 19.220935066573706
927300 | 19.220555912594367
927400 | 19.220177876927895
927500 | 19.219777612323245
927600 | 19.219401704347096
927700 | 19.21903581876632
927800 | 19.21865436700921
927900 | 19.218282157530442
928000 | 19.217899744341825
928100 | 19.217531252363134
928200 | 19.217144007815254
928300 | 19.216757138665916
928400 | 19.216362149980117
928500 | 19.215974360555496
928600 | 19.21556543234738
928700 | 19.21516160624947
928800 | 19.21476870377251
928900 | 19.214375842015446
929000 | 19.21396211126535
929100 | 19.21354768245465
929200 | 19.213140825912664
929300 | 19.21273840897243
929400 | 19.212311783614727
929500 | 19.211926203520083
929600 | 19.21151097424536
929700 | 19.211106548259526
929800 | 19.210699549048517
929900 | 19.210285558003083
930000 | 19.20985947854979
930100

956200 | 19.12421257299434
956300 | 19.123837215847065
956400 | 19.123473937697103
956500 | 19.123110731867886
956600 | 19.122745525052554
956700 | 19.122396860025905
956800 | 19.12202150670669
956900 | 19.121670327572023
957000 | 19.12129492560121
957100 | 19.12093465075191
957200 | 19.120562994719652
957300 | 19.120204533024438
957400 | 19.119839011514845
957500 | 19.119480120386246
957600 | 19.119112742910833
957700 | 19.118757480503294
957800 | 19.118396622068534
957900 | 19.118043849932523
958000 | 19.117679096654914
958100 | 19.11731678308411
958200 | 19.11695315998873
958300 | 19.116579087263975
958400 | 19.116193050949963
958500 | 19.115789607443052
958600 | 19.115402098082317
958700 | 19.115018197529256
958800 | 19.114610860859333
958900 | 19.11420163244475
959000 | 19.113784175194954
959100 | 19.11338065298401
959200 | 19.11297853776637
959300 | 19.112560822507707
959400 | 19.112147532535577
959500 | 19.111718255458758
959600 | 19.111293502113543
959700 | 19.11086951735638
95

985900 | 18.998488894812002
986000 | 18.99818172286405
986100 | 18.99788329980402
986200 | 18.99757799853774
986300 | 18.997281216345304
986400 | 18.996970088732727
986500 | 18.99668917555437
986600 | 18.99638941498461
986700 | 18.996097708543633
986800 | 18.995801212802466
986900 | 18.99551573499389
987000 | 18.995213507203292
987100 | 18.99494345829726
987200 | 18.99465138943788
987300 | 18.99435255007211
987400 | 18.994071751386404
987500 | 18.993788246888474
987600 | 18.993492451888645
987700 | 18.993205950769383
987800 | 18.992905075905814
987900 | 18.99260579271345
988000 | 18.992302120779872
988100 | 18.99199107066511
988200 | 18.991696224494493
988300 | 18.99138619131168
988400 | 18.991063684728264
988500 | 18.990765877396147
988600 | 18.990446381137193
988700 | 18.990144199289947
988800 | 18.98982689071511
988900 | 18.989523143282558
989000 | 18.989204001147076
989100 | 18.988885222594323
989200 | 18.98856551222889
989300 | 18.988270655554462
989400 | 18.98796126922156
989500 

In [10]:
output

tensor([[ 0.0089, -0.0154,  0.0032,  ...,  0.0555,  0.0424,  0.0597],
        [ 0.0085, -0.0146,  0.0032,  ...,  0.0557,  0.0412,  0.0591],
        [ 0.0086, -0.0150,  0.0038,  ...,  0.0566,  0.0415,  0.0575],
        ...,
        [ 0.0079, -0.0147,  0.0040,  ...,  0.0564,  0.0412,  0.0584],
        [ 0.0080, -0.0148,  0.0038,  ...,  0.0560,  0.0415,  0.0586],
        [ 0.0090, -0.0155,  0.0040,  ...,  0.0561,  0.0412,  0.0599]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [10]:
output[0]

tensor([-0.0325, -0.0138,  0.0427,  ...,  0.0129, -0.0514, -0.0116],
       device='cuda:0', grad_fn=<SelectBackward>)

In [11]:
input[0] - output[0]

tensor([ 0.0325,  0.0138, -0.0427,  ..., -0.0459,  0.0150, -0.0287],
       device='cuda:0', dtype=torch.float64, grad_fn=<SubBackward0>)

In [12]:
def calcLoss(original_data, processed_data):
    loss = zip(original_data, processed_data)
    loss = list(map(lambda x: x[0] - x[1], loss))
    loss = list(map(lambda x: reduce(lambda a, b: a + b, x), loss))
    loss = reduce(lambda a, b: a + b, loss)
    return abs(loss)

In [13]:
calcLoss(input, output)

tensor(132.9474, device='cuda:0', dtype=torch.float64, grad_fn=<AbsBackward>)

In [12]:
torch.save(net.state_dict(), 'net.pt')